In [1]:
import ray
import pandas
import requests

from typing import Dict

In [2]:
# Configurate libraries
_ = ray.init()

2019-12-15 09:47:28,265	INFO resource_spec.py:205 -- Starting Ray with 1.86 GiB memory available for workers and up to 0.93 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


In [11]:
# This are the paramenters used for the script
base_url = "http://sicorresultadosapi.integ.ro"

auth_end_point = "/login"
api_username = ""
api_password = ""

map_meta_end_point = "/get_mapa"
map_data_end_point = "/MAPA"
medicaments_end_points = "/medicamentos"

start_patient_id = 12760
end_patient_id = 12766

In [12]:
auth_url = f"{base_url}{auth_end_point}"
map_meta_url = f"{base_url}{map_meta_end_point}"
map_data_url = f"{base_url}{map_data_end_point}"
medicaments_url = f"{base_url}{medicaments_end_points}"

In [13]:
def authenticate_api(url: str, username: str, password: str) -> Dict:
    payload = {
        "user": username,
        "password": password,
    }
    
    response = requests.post(url, data=payload)
    
    return response.json()['res']

In [14]:
def get_data(url:str, token: str, patient_id: int) -> Dict:
    url = (url + "/{patient_id}/").format(patient_id=patient_id)
   
    response = requests.get(
        url,
        headers = {
            "authorization": f"Bearer {token}",
        },
    )
    
    return response.json()

In [15]:
@ray.remote
def get_data_async(username: str, password: str, patient_id: int) -> Dict:
    token = authenticate_api(auth_url, username, password)
    
    map_meta_data = get_data(map_meta_url, token, patient_id)
    map_data = get_data(map_data_url, token, patient_id)
    meds_data = get_data(medicaments_url, token, patient_id)
    
    return {
        'map_meta_data': map_meta_data,
        'map_data': map_data,
        'medicaments_url': meds_data,
    }

In [16]:
futures = [get_data_async.remote(api_username, api_password, patient_id) for patient_id in range(start_patient_id, end_patient_id)]
api_data = ray.get(futures)